In [1]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv("Farms_Yellowpages.csv")

In [10]:
df

,Unnamed: 0,Name,Adress,Phone,Website,Email,Source,DateUpdated,DateChecked
0,0,The Farm Byron Bay,"11 Ewingsdale Rd, Ewingsdale NSW 2481",(02) 6684 7888,http://www.thefarmbyronbay.com.au/,info@thefarmbyronbay.com.au,https://www.yellowpages.com.au/search/listings...,19/04/2020,-
1,1,GCS Partners In Farming,"15 / 553 New Grenfell Rd, Forbes NSW 2871",0409 364 382,N/V,gcmoxey@bigpond.com,https://www.yellowpages.com.au/search/listings...,19/04/2020,-
2,2,Boer Bok Goat Stud,"PO Box 537, Busselton WA 6280",(08) 9753 1160,http://www.boerbok.com.au/,bethh@boerbok.net.au,https://www.yellowpages.com.au/search/listings...,19/04/2020,-
3,3,Kilsby Malcolm & Karen,"PO Box 458, Mt Gambier SA 5290",(08) 8738 5322,N/V,N/V,https://www.yellowpages.com.au/search/listings...,19/04/2020,-
4,4,Baboo Pastoral Co,"""Wandija"", Green Range WA 6328",(08) 9846 6031,N/V,N/V,https://www.yellowpages.com.au/search/listings...,19/04/2020,-
5,5,Correll Ag Group,Port Clinton SA 5570,(08) 8837 7018,N/V,N/V,https://www.yellowpages.com.au/search/listings...,19/04/2020,-
6,6,Kurana Farms,"325 Lillimur Rd North, Kaniva VIC 3419",(03) 5392 2747,http://www.kuranafarms.com.au/,bnjewell@bigpond.com,https://www.yellowpages.com.au/search/listings...,19/04/2020,-
7,7,Robert J Clarke,N/V,(03) 6392 2279,http://www.rjclarke.com.au/,robandheatherclarke@bigpond.com,https://www.yellowpages.com.au/search/listings...,19/04/2020,-
8,8,DP & NA Kerr,N/V,(03) 5593 1287,http://www.dpandnakerr.com.au/,relsy75@hotmail.com,https://www.yellowpages.com.au/search/listings...,19/04/2020,-
9,9,PM and SC Nixon,N/V,(02) 6742 4497,http://www.pmscnixonhaymaking.com.au/,paulnixon49@bigpond.com,https://www.yellowpages.com.au/search/listings...,19/04/2020,-


In [11]:
def cleanup(df):
    return df.drop(labels="Unnamed: 0", axis=1)

def get_zip(df):
    zip_codes = list()
    state = list()
    for i in range(0, len(df.Name)):
        zip_code = df.Adress[i][-4:]
        zip_codes.append(zip_code)
        
        try:
            intzip_code = int(zip_code)
            if intzip_code >= 1000 and intzip_code < 3000:
                state.append("NSW")
            elif (intzip_code >= 3000 and intzip_code < 4000) or (intzip_code >= 8000 and intzip_code < 9000):
                state.append("Victoria")
            elif (intzip_code >= 4000 and intzip_code < 5000) or (intzip_code >= 9000 and intzip_code < 10000):
                state.append("Queensland")
            elif intzip_code >= 5000 and intzip_code < 6000:
                state.append("South Australia")
            elif intzip_code >= 6000 and intzip_code < 7000:
                state.append("Western Australia")
            elif intzip_code >= 7000 and intzip_code < 8000:
                state.append("Tasmania")
            elif intzip_code >= 800 and intzip_code < 1000:
                state.append("Nothern Australia")
            else:
                state.append("N/V")
        except:
            state.append("N/V")
            
            
    df["zip"] = zip_codes
    df["state"] = state
    return df

In [12]:
df = cleanup(df)

In [13]:
df = get_zip(df)